1. Data Scrapping

>using webscraper.io to get the data from https://www.metacritic.com/publication/ign/?sort-options=date        
then using IGDB API to fetch the game's genre

In [1]:
import requests
import pandas as pd
import logging

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Your IGDB API credentials
CLIENT_ID = '6wh19hcnh8whtcrkmigpn6z97nsu11'
CLIENT_SECRET = 'vxkuabmeskb17dj8eg9w7m2btg44d7'

# Function to get new access token
def get_access_token(client_id, client_secret):
    auth_response = requests.post('https://id.twitch.tv/oauth2/token', data={
        'client_id': client_id,
        'client_secret': client_secret,
        'grant_type': 'client_credentials'
    })
    auth_data = auth_response.json()
    return auth_data['access_token']

# Get new access token
ACCESS_TOKEN = get_access_token(CLIENT_ID, CLIENT_SECRET)

headers = {
    'Client-ID': CLIENT_ID,
    'Authorization': f'Bearer {ACCESS_TOKEN}'
}

# Function to get game genre from IGDB
def get_game_genre(game_title):
    query = f'search "{game_title}"; fields name, genres.name;'
    response = requests.post('https://api.igdb.com/v4/games', headers=headers, data=query)
    
    if response.status_code == 200:
        data = response.json()
        if data:
            genres = data[0].get('genres', [])
            genre_names = [genre['name'] for genre in genres]
            logging.info(f"Fetched genre for '{game_title}': {genre_names}")
            return ', '.join(genre_names)
        else:
            logging.warning(f"No data found for '{game_title}'")
            return 'Unknown'
    else:
        logging.error(f"Failed to fetch genre for '{game_title}'. Status code: {response.status_code}")
        return 'Error'

# Read the CSV file
csv_file = 'metacritic.csv'  
df = pd.read_csv(csv_file)

# Fetch genres for each game and update the DataFrame
df['genre'] = df['title'].apply(get_game_genre)

# Save the updated DataFrame back to CSV
df.to_csv(csv_file, index=False)

print("CSV file updated with game genres.")


2025-03-16 17:38:39,140 - INFO - Fetched genre for 'SteamWorld Heist II': ['Role-playing (RPG)', 'Strategy', 'Adventure', 'Indie']
2025-03-16 17:38:39,586 - INFO - Fetched genre for 'Earth Defense Force 6': ['Shooter']
2025-03-16 17:38:40,359 - INFO - Fetched genre for 'Star Wars Outlaws': ['Adventure']
2025-03-16 17:38:41,085 - INFO - Fetched genre for 'Star Wars Outlaws': ['Adventure']
2025-03-16 17:38:41,784 - INFO - Fetched genre for 'EA Sports College Football 25': ['Simulator', 'Sport']
2025-03-16 17:38:42,134 - INFO - Fetched genre for 'Nobody Wants to Die': ['Role-playing (RPG)', 'Simulator', 'Adventure']
2025-03-16 17:38:42,802 - INFO - Fetched genre for 'F1 Manager 2024': ['Racing', 'Simulator', 'Sport', 'Strategy']
2025-03-16 17:38:43,501 - INFO - Fetched genre for 'Nintendo World Championships: NES Edition': ['Platform', 'Racing']
2025-03-16 17:38:43,922 - INFO - Fetched genre for 'Flintlock: The Siege of Dawn': ['Role-playing (RPG)', 'Adventure', 'Indie']
2025-03-16 17:38:

KeyboardInterrupt: 

2. Data Cleaning

- ignore NaN and tbd values from rating
- convert score rating to 0-10
- adding score phrase variable
- convert datetime and adding year, month, day variable
- remove unnecessary variable

In [2]:
import pandas as pd

# Load the CSV file
df = pd.read_csv('metacritic.csv')

# Rename the 'rating' column to 'score'
df = df.rename(columns={'rating': 'score'})

# Filter out rows where the rating is 'tbd' or NaN values
df = df[df['score'] != 'tbd']
df = df.dropna(subset=['score'])

# Convert rating to 10-point scale
df['score'] = df['score'].astype(int) // 10

# Create a dictionary to map ratings to score phrases
score_phrases = {
    10: 'Masterpiece', 9: 'Amazing', 8: 'Great', 7: 'Good', 6: 'Okay',
    5: 'Mediocre', 4: 'Bad', 3: 'Awful', 2: 'Painful', 1: 'Unbearable'
}
df['score_phrase'] = df['score'].map(score_phrases)

# Convert release_date to datetime format
df['release_date'] = pd.to_datetime(df['release_date'])

# Create new variables for year, month, and day
df['release_year'] = df['release_date'].dt.year
df['release_month'] = df['release_date'].dt.month
df['release_day'] = df['release_date'].dt.day

# Drop unnecessary columns
df.drop(columns=['web-scraper-order', 
        'web-scraper-start-url', 
        'release_date'], inplace=True)

# Save the cleaned data to a new CSV file
df.to_csv('cleaned_data.csv', index=False)

print(df)

                               title       platform  score  \
0                SteamWorld Heist II             PC      8   
1              Earth Defense Force 6  PlayStation 5      7   
4      EA Sports College Football 25  Xbox Series X      7   
5                Nobody Wants to Die             PC      7   
6                    F1 Manager 2024  PlayStation 5      8   
...                              ...            ...    ...   
2504                         Mad Max  PlayStation 4      8   
2505     Disney Infinity 3.0 Edition  PlayStation 4      8   
2506     Disney Infinity 3.0 Edition       Xbox One      8   
2507  Dishonored: Definitive Edition  PlayStation 4      8   
2508  Dishonored: Definitive Edition       Xbox One      8   

                                               genre score_phrase  \
0     Role-playing (RPG), Strategy, Adventure, Indie        Great   
1                                            Shooter         Good   
4                                   Simulator, S

3. Merging Data


In [3]:
import pandas as pd

csv1 = pd.read_csv('IGN games from best to worst.csv')
csv2 = pd.read_csv('cleaned_data.csv')

merged_df = pd.concat([csv1, csv2], 
                ignore_index=True).drop_duplicates(keep='first')
merged_df = merged_df.sort_values(by='score', ascending=False)

merged_df.to_csv('merged_data.csv', index=False)

print(merged_df)

                                         title  score score_phrase  \
0                               Checkered Flag   10.0  Masterpiece   
49                                 The Witness   10.0  Masterpiece   
19279             Disco Elysium: The Final Cut   10.0  Masterpiece   
19278             Disco Elysium: The Final Cut   10.0  Masterpiece   
19126                                Deathloop   10.0  Masterpiece   
...                                        ...    ...          ...   
18601                      High Rollers Casino    1.0   Unbearable   
18619                                  Step Up    1.0   Unbearable   
18622                      Action Girlz Racing    0.8     Disaster   
18623                       Extreme PaintBrawl    0.7     Disaster   
18624  Looney Tunes: Back in Action: Zany Race    0.5     Disaster   

            platform                                 genre  release_year  \
0               Lynx                                Racing          1999   
49     